### Install Required Libraries

In [ ]:
# Install necessary libraries
!pip install transformers accelerate git+https://github.com/facebookresearch/detr.git -q
!pip install torch torchvision -q


ERROR: git+https://github.com/facebookresearch/detr.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.8 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
# Mount Google Drive to access image folder and CSV file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import Python Libraries

In [ ]:
# Import necessary libraries
from transformers import ViltProcessor, ViltForQuestionAnswering
from PIL import Image
import torch
import pandas as pd
import os
import re
from tqdm import tqdm


### Load VILT Processor and Model

In [ ]:
# Setup device for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load processor and model for VILT QnA
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/136k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/470M [00:00<?, ?B/s]

### Load CSV and Format Questions

In [ ]:
# Load your CSV from Google Drive
csv_file_path = '/content/drive/MyDrive/VR_proj2/proper_result_2_final.csv'  # Modify path accordingly
df = pd.read_csv(csv_file_path)

# Add constraint to each question for one-word answer
df['question'] = df['question'].apply(lambda q: f"Answer in one word: {q.strip()}")

# Save back or to a new CSV
df.to_csv("/content/drive/MyDrive/VR_proj2/test_1_oneword.csv", index=False)
print("Updated CSV saved as test_1_oneword.csv with 'Answer in one word' prompt.")


Updated CSV saved as test_1_oneword.csv with 'Answer in one word' prompt.


###  Run VILT Model on Each Image-Question Pair

In [ ]:
# Set row range for batch processing
start_idx = 50000
end_idx = 60000  # exclusive, so rows 0 to 19
subset_df = df.iloc[start_idx:end_idx]
results = []

# Process each row
for idx, row in tqdm(subset_df.iterrows(), total=len(subset_df), desc="Running VILT on selected rows"):
    image_path = os.path.join("/content/drive/MyDrive/VR_proj2/abo-images-small", row['path'])
    if not os.path.exists(image_path):
        results.append("image_not_found")
        continue

    try:
        # Load image and question
        image = Image.open(image_path).convert("RGB")
        question = row["question"]

        # Prepare inputs
        inputs = processor(images=image, text=question, return_tensors="pt").to(device)

        # Forward pass to get logits
        with torch.no_grad():
            outputs = model(**inputs)

        # Get the logits and predicted token IDs
        logits = outputs.logits
        idx = logits.argmax(dim=-1).item()

        # Use id2label mapping to get the answer
        answer = model.config.id2label[idx]

        # Handle cases where the answer might be empty or invalid
        if not answer:
            answer = "no_answer"
        answer = re.sub(r"[^\w]", "", answer)  # Remove punctuation

    except Exception as e:
        answer = f"error: {str(e)}"

    results.append(answer)

# Save results to a new CSV
subset_df['vilt_answer'] = results
subset_df.to_csv(f"/content/drive/MyDrive/VR_proj2/vilt_vqa_results_{start_idx}_{end_idx}.csv", index=False)


Running VILT on selected rows: 100%|██████████| 9847/9847 [23:38<00:00,  6.94it/s]
<ipython-input-10-0d84288c0dcf>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['vilt_answer'] = results
